In [1]:
import sqlite3 as sql
from datetime import datetime

In [3]:
from abc import ABC
from enum import Enum

In [3]:
database = "foodOrder"
connection = sql.connect(database)

In [4]:
connectDB = connection.cursor()

In [13]:
class DataAccessor:
    def getMatchingMealNames(self, foodItem):
        pass
    
    def addItemsToCart(self, user, foodItem):
        pass
    
    def getUserCart(self, user):
        pass
    
    def deletItemsFromCart(self, user, foodItem):
        pass
    
    def checkoutUserCart(self, user):
        pass
    
    def createOrder(self, user, cartItems):
        pass

In [6]:
class MealType(Enum):
    Veg = 1,
    NonVeg = 2
    
    def getMealType(mealType):
        try:
            meal=MealType[mealType].value[0]
            return True, meal
        except:
            return False, -1

In [7]:
class Cuisine(Enum):
    SouthIndian = 1,
    Mughlai = 2,
    Italian = 3,
    Punjabi = 4,
    American = 5,
    Mexican = 6,
    Chinese = 7,
    Japanese = 8,
    Korean = 9,
    French = 10
    
    def getCuisine(cuisineType):
        try:
            cuisine=Cuisine[cuisineType]
            cuisineValue=cuisine.value[0]
            cuisneName=cuisine.name[0]
            return True, cuisineValue, cuisineName
        except:
            return False, -1, None

In [8]:
class Rating(Enum):
    one = 1,
    two = 2,
    three = 3,
    four = 4,
    five = 5
    
    def getRating(rating):
        try:
            rate=Rating[rating].value[0]
            return True, rate
        except:
            return False, -1

In [20]:
class PaymentStatus(Enum):
    success = 1,
    failed = 2
    
    def getPaymentStatus(status):
        try:
            status=PaymentStatus[status].value[0]
            return  True, status
        except:
            return False, -1

In [4]:
class PaymentMode(Enum):
    netBanking = 1,
    UPI = 2,
    card = 3,
    cashOnDelivery = 4
    
    def getPaymentMode(mode):
        try:
            mode = PaymentMode[mode].value[0]
            return True, mode
        except:
            return False, -1

In [6]:
class OrderStatus(Enum):
    orderPlaced = 1,
    cooking = 2,
    readyForDelivery = 3,
    outForDelivery = 4, 
    Delivered = 5,
    cancelled = 6
    
    def getOrderStatus(status):
        try:
            status = OrderStatus[status].value[0]
            return True, status
        except:
            return False, -1

In [9]:
class Address:
    def __init__(self, addressLine1, addressLine2, city, pinCode, state):
        self.addressLine1 = addressLine1
        self.addressLine2 = addressLine2
        self.city = city
        self.pinCode = pinCode
        self.state = state

In [10]:
class BusinessHours:
    def __init__(self, openingTime, closingTime):
        self.openingTime = self.getTimeFromString(openingTime)
        self.closingTime = self.getTimeFromString(closingTime)
        
    def getTimeFromString(self, datetimeStr):
        datetimeStr = datetimeStr

        datetimeStr = datetimeStr.split(' ')
        if(datetimeStr[-1]=='PM' and datetimeStr[0]!='12'):
            hour = int(datetimeStr[0])+12
            datetimeStr = str(hour)+':'+'00'+':'+'00'
        else:
            if(datetimeStr[0]=='12'):
                datetimeStr[0]=='00'
            datetimeStr = datetimeStr[0]+':'+'00'+':'+'00'

        datetimeObject = datetime.strptime(datetimeStr, '%H:%M:%S')
        
        return datetimeObject.hour
    
    def displayDateTime(self):
        unit='AM' if(self.openingTime<11) else 'PM'
        hour = self.openingTime if(1<=self.openingTime<=12) else abs(self.openingTime-12) 
        openTime = str(openingTime) + unit
        
        unit='AM' if(self.closingTime<11) else 'PM'
        hour = self.closingTime if(1<=self.closingTime<=12) else abs(self.closingTime-12) 
        closeTime = str(closeTime) + unit
        
        
        return openTime+" - "+closeTime

In [19]:
class PaymentResponse:
    def __init__(self, id, amount, paymentStatus):
        self.id=id
        self.amount=amount
        self.paymentStatus = paymentStatus

In [11]:
class FoodItem:
    def __init__(self, id, name, description, price, mealType, cuisineType, ratings, restaurantID, isAvalaible):
        self.id  = id
        self.name = name
        self.description = description
        self.price = price
        self.mealType = MealType.getMealType(mealType)
        self.cuisineType = Cuisine.getCuisine(cuisineType)[1]
        self.ratings = Rating.getRating(ratings)
        self.restaurantID = restaurantID
        self.isAvailable = isAvalaible
    
    def getSearchFilteredVars(self):
        return [self.mealType, self.cuisineType, self.ratings]

In [12]:
class Restaurant:
    def __init__(id, name, description, businessHours, mealType, cuisineType, rating, menu, address):
        self.id = id
        self.name = name
        self.description = description
        self.businessHours = [BusinessHours().openingTime, "-", BusinessHours.closingTime]
        self.mealType = MealType.getMealType(mealType)
        self.cusineType = self.CuisineTypeFilter.returnMultipleItems(cuisineType)
        self.rating = Rating.getRating(rating)
        self.menu = Menu.getMenu(menu)
        self.address = Address()

In [13]:
class User:
    def __init__(self,id,name,address,phoneNumber,email):
        self.id = id
        self.name = name
        self.address = Address()
        self.phoneNumber = phoneNumber
        self.email = email

In [14]:
class Menu:
    def __init__(self, menuList):
        self.menu=menuList
    
    def getMenuList(self):
        return self.menu

In [1]:
class Order:
    def __init__(self, cartItems, orderId, userId, orderStatus):
        self.cartItems=cartItems
        self.orderId=orderId
        self.userId=userId
        self.orderStatus=orderStatus

In [8]:
class Permission(ABC):
    def isPermitted(self):
        pass

In [51]:
class AddToCartPermission(Permission):
    user = User('id', 'name', 'address', 'phoneNumber', 'email')
    foodItem = FoodItem('id', 'name', 'description', 'price', 'mealType', 'cuisineType', 'ratings', 'restaurantID', 'isAvalaible')
    deliveryManager = DeliveryManager() 
    
    def isPermitted(self, user, foodItem):
        if not foodItem.isAvailable():
            return False
        restaurant = RestaurantSearcher().searchById(foodItem.restaurantID())
        return deliveryManager.isDeliveryPossible(user.address, restaurant.address)

In [52]:
class CheckOutCartPermission(Permission):
    user = User()
    
    def isPermitted(self, user):
        pass

In [53]:
class DeleteFromCartPermission(Permission):
    user = User()
    foodItem = FoodItem()
    
    def isPermitted(self, user, foodItem):
        pass

In [54]:
class UpdateCartPermission(Permission):
    user = User()
    foodItem = FoodItem()
    
    def isPermitted(self, user, foodItem):
        pass

In [57]:
class ProceedToCheckoutPermission(Permission):
    user = User()
    
    def isPermitted(self, user):
        pass

In [10]:
class PlacedOrderPermission(Permission):
    user = User()
    
    def isPermitted(self, user):
        pass

In [21]:
class UpdateOrderPermission(Permission):
    user = User()
    
    def isPermitted(self, user):
        pass

In [16]:
class FoodItemFilter(ABC):
    def filter(self, Item):
        pass

class RestaurantFilter(ABC):
    def filter(self, restaurant):
        pass
    
    def returnMultipleItems(self, cuisineList):
        pass

In [17]:
class MealTypeFilter(FoodItemFilter, RestaurantFilter):
    def filter(self, mealType):
        meals=MealType.getMealType(mealType)
        if not meals[0]:
            return None
        else:
            return meals[1]
        
class CuisineTypeFilter(FoodItemFilter, RestaurantFilter):
    def filter(self, cuisineType):
        cuisines=Cuisine.getCuisine(cuisineType)
        if not cuisines[0]:
            return None
        else:
            return cuisines[1]
    
    def returnMultipleItems(self, cuisineList):
        cuisinesAvaialable=[]
        for cuisine in cuisineList:
            cuisinesAvaialable.append(Cuisine.getCuisine(cuisine).name)

class RatingFilter(FoodItemFilter, RestaurantFilter):
    def filter(self, ratingType):
        ratings=Rating.getRating(ratingType)
        if not ratings[0]:
            return None
        else:
            return ratings[1]

In [19]:
class FoodItemSearchAPI:
    def searchFoodItems(self, foodItemName, mealType, cuisineType, ratings):
        return FoodItemSearcher().search(foodItemName, [MealTypeFilter.filter(mealType), CuisineTypeFilter.filter(cuisineType),RatingFilter.filter(ratings)])
class RestaurantSearchAPI:
    def searchRestaurants(self, restaurantName, mealType, cuisineType, ratings):
        return RestaurantSearcher().search(restaurantName, [MealTypeFilter.filter(mealType), CuisineTypeFilter.filter(cuisineType),RatingFilter.filter(ratings)])
class AddToCartAPI:
    userManager = UserManager()
    foodItemSearcher = FoodItemSearcher()
    cartManger = CartManager()
    
    def addToCart(self, authToken, foodItemId):
        if(authToken is None or not authToken or foodItemId<0):
            pass
        user = UserManager.getUserByToken(authToken)
        if user is None:
            pass
        foodItem = FoodItemSearcher.searchFoodById(foodItemId)
        if foodItem is None:
            pass
        CartManger().addItemsToCart(user, foodItem)
        
class PlaceOrderAPI:
    userManager = UserManager()
    orderManager = OrderManager()

    def placeOrder(self, authToken, paymentInfo, paymentMode):
        if not authToken:
            return None
        
        user = userManager.getUserByToken(authToken)
        if not user:
            return None
        
        paymentManager = PaymentManagerFactory.getPaymentManager(paymentMode, payemntInfo)
        paymentResponse = paymentManger.executePayment()
        if not paymentResponse:
            raise Exception("Payment Failed")
        
        orderManager.placeOrder(user)
        
class UpdateOrderAPI:
    userManager = UserManager()
    orderManager = OrderManager()
    
    def updateOrder(self, orderId, status, authToken):
        if not self.authToken:
            return None
        
        user = userManager.getUserByToken(authToken)
        if not user:
            return None
        
        order = OrderManager.getOrder(self.orderId)
        if not order:
            return None
        

In [67]:
class CartManager:
    def getUserCart(self, user):
        return DataAccessor().getUserCart(user)
    
    def addItemsToCart(self, user, foodItem):
        permission = PermissionFactory().getAddToCartPermission(user, foodItem)
        if not permission.isPermitted():
            raise Exception("Permission Denied!")
        if not self.isFoodFromSameRestaurant(user, foodItem):
            raise Exception("Food Item is not from Same Restaurant")
        DataAccesor().addItemsToCart(user, foodItem)
        
    def deleteItemFromCart(self, user, foodItem):
        permission = PermissionFactory().getDelteFromCartPermission(user, foodItem)
        if not permission:
            raise Exception("Permission Denied!")
        if not self.isFoodItemPresentInCart(user, foodItem):
            raise Exception("Permission Denied!")
        DataAccessor().deleteItemsFromCart(user, foodItem)
        
    def checkOutUserCart(self, user):
        permission = PermissionFactoty().getCheckOutUserCarrt(user)
        if not permission:
            raise Exception("Permission Denied!")
        if self.isUserCartEmpty(user):
            raise Exception("Permission Denied!")
        DataAccessor().checkoutUserCart(user)
    
    def isFoodFromSameRestaurant(self, user, foodItem):
        userCart = self.getUserCart(user)
        if not userCart:
            return True
        for item in userCart:
            if item.getFoodItem().restaurantId != foodItem.restaurantId:
                return False
        return True
    
    def isFoodItemPresentInCart(self, user, foodItem):
        userCart = self.getUserCart(user)
        if not userCart:
            return True
        for item in userCart:
            if item.getFoodItem().id == foodItem.id:
                return True
        return False
    
    def isUserCartEmpty(self, user):
        userCart = self.getUserCart(user)
        if not userCart:
            return True
        

In [21]:
class PaymentManager(ABC):
    def executePayment(self):
        pass

In [22]:
class NetBankingPaymentManager(PaymentManager):
    def __init__(self, bankName, userName, password, pin, amount):
        self.bankName = bankName
        self.userName = username
        self.password = password
        self.pin = pin
        self.amount = amount
    
    def executePayment(self):
        pass

In [23]:
class CardBasedPaymentManager(PaymentManager):
    def __init__(self, bankName, cardNumber, pin, amount):
        self.bankName = bankName
        self.cardNumber = cardNumber
        self.pin = pin
        self.amount = amount
    
    def executePayment(self):
        pass

In [26]:
class UPIPaymentManager(PaymentManager):
    def __init__(self, upiID, upiPin, amount):
        self.upiID = upiID
        self.upiPin = upiPin
        self.amount = amount
    
    def executePayment(self):
        pass

In [27]:
class CashOnDeliveryPaymentManager(PaymentManager):
    def __init__(self, amount):
        self.amount = amount
    
    def executePayment(self):
        pass

In [29]:
class UserManager:
    def getUserByToken(self, authToken):
        pass

In [22]:
class OrderManager:
    def placeOrder(self, user):
        permission = PermissionFactory().getPlacedOrderPermission(self.user)
        if not permission:
            raise Exception("Invalid!")
        cartManager = CartManager()
        carManager.getUserManager(self.user)
        orderId = DataAccessor.createOrder(self.user, self.cartItems)
        cartManager().checkOutUser(self.user)
        return Order(cartItems, orderId, self.user.getId, OrderStatus['orderPlaced'])
    
    def getOrders(self, user):
        pass
    
    def getOrder(self, user):
        pass
    
    def setOrderToCooking(self, user, order):
        permission = PermissionFactory().getUpdateOrderPermission(self.user, self.order, OrderStatus['cooking'])
        if not permission:
            raise Exception("Invalid")
        if not self.order.getOrderStatus==OrderStatus['orderPlaced']:
            raise Exception('Only Order Placed can be moved')
    
    def setOrderToCooking(self, user, order):
        permission = PermissionFactory().getUpdateOrderPermission(self.user, self.order, OrderStatus['cooking'])
        if not permission:
            raise Exception("Invalid")
        if not self.order.getOrderStatus==OrderStatus['orderPlaced']:
            raise Exception('Only Order Placed can be moved')
    
    def setOrderToCooking(self, user, order):
        permission = PermissionFactory().getUpdateOrderPermission(self.user, self.order, OrderStatus['cooking'])
        if not permission:
            raise Exception("Invalid")
        if not self.order.getOrderStatus('orderPlaced')!=OrderStatus['orderPlaced']:
            raise Exception('Only Order Placed can be moved')
    
    def setOrderToReadyForDelivery(self, user, order):
        permission = PermissionFactory().getUpdateOrderPermission(self.user, self.order, OrderStatus['ReadyForDelivery'])
        if not permission:
            raise Exception("Invalid")
        if not self.order.getOrderStatus('orderPlaced')!=OrderStatus['orderPlaced']:
            raise Exception('Only Order Placed can be moved')
    
    def setOrderToOutForDelivery(self, user, order):
        permission = PermissionFactory().getUpdateOrderPermission(self.user, self.order, OrderStatus['outForDelivery'])
        if not permission:
            raise Exception("Invalid")
        if not self.order.getOrderStatus('orderPlaced')!=OrderStatus['orderPlaced']:
            raise Exception('Only Order Placed can be moved')
    
    def setOrderToCancelled(self, user, order):
        permission = PermissionFactory().getUpdateOrderPermission(self.user, self.order, OrderStatus['cancelled'])
        if not permission:
            raise Exception("Invalid")
        if not self.order.getOrderStatus('orderPlaced')!=OrderStatus['orderPlaced']:
            raise Exception('Only Order Placed can be moved')


In [35]:
class DeliveryManager:
    def isDeliveryPossible(self, userAddress, restaurantAddress):
        return True

In [20]:
class CartItem:
    def __init__(self, foodItem, quantity):
        self.foodItem=foodItem
        self.quantity=quantity

In [24]:
class FoodItemSearcher:
    def search(self, foodItem, filters):
        if len(foodItem)==0 or foodItem is None:
            return "No Parameters Provided"
        foodItems=DataAccessor().getMatchingMealNames(foodItem)
        FinalFoodList={}
        for food in foodItems:
            finalFoodList[food.id]=[]
            passFood=0
            if food.getSearchFilteredVars()==filters:
                finalFoodList[food.id].append(food.name)
        
        return finalFoodList
    
    def searchFoodById(self, foodItemId):
        pass
                

In [34]:
class RestaurantSearcher:
    def search(self, restaurantName, mealType, cuisineType, rating):
        result=self.searchByName(restaurantName)[0]
        if result[0]:
            return result[1]
        else:
            filters=[]
            if mealType[0]:
                filters.append(mealType[1])
            else:
                filters.append(None)
            if cuisineType[0]:
                filters.append(cuisineType[1])
            else:
                filters.append(None)
            if rating[0]:
                filters.append(rating[1])
            else:
                filters.append(None)
            
            return self.searchByFilters(filters)
    
    def searchByName(self, restaurantName):
         return DataAccessor().getMatchingRestaurantNames(restaurantName)
    
    def searchByFilters(self, filters):
        return list
    
    def searchById(self, restaurantId):
        pass

In [20]:
class PermissionFactory:
    def getAddToCartPermission(self, user, foodItem):
        addToCartPermission=AddToCartPermission(self.user, self.foodItem)
        return addToCartPermission
    
    def getDelteFromCartPermission(self, user, foodItem):
        deleteFromCartPermission=DeleteFromCartPermission(self.user, self.foodItem)
        return deleteFromCartPermission
        
    def getUpdateCartPermission(self, user, foodItem):
        updateCartPermission=UpdateCartPermission(self.user, self.foodItem)
        return updateCartPermission
    
    def getCheckOutCartPermission(self, user):
        checkoutCartPermission=CheckOutCartPermission(user)
        return checkoutCartPermission
    
    def getProceedToCheckoutPermission(self, user):
        proceedToCheckoutPermission=ProceedToCheckoutPermission(self.user)
        return proceedToCheckoutPermission
    
    def getPlacedOrderPermission(self, user):
        placedOrderPermission=PlacedOrderPermission(self.user)
        return placedOrderPermission
    
    def getUpdateOrderPermission(self, user):
        updateOrderPermission = UpdateOrderPermission(self.user)
        return updateOrderPermission

In [29]:
class PaymentManagerFactory:
    def getPaymentManager(self, paymentMode, paymentInfo):
        PaymentManager = None
        
        payment = PaymentMode.getPaymentMode(paymentMode)
        if(payment == 1):
            self.getNetBankingPaymentManager(paymentInfo)
        elif(payment == 2):
            self.getUPIPaymentManager(paymentInfo)
        elif(payment == 3):
            self.getCardPaymentManager(paymentInfo)
        elif(payment == 4):
            self.getCashOnDeliveryPaymentManager(paymentInfo)
        else:
            raise Exception("Invalid Payment Method")
        
    def getNetBankingPaymentManager(self, paymentInfo):
        bankName = paymentInfo['bankName']
        userName = paymentInfo['paymentInfo']
        password = paymentInfo['password']
        pin = paymentInfo['pin']
        amount = paymentInfo['amount']
        
        return NetBankingPaymentManager(bankName, userName, password, pin, amount)
     
    def getUPIPaymentManager(self, paymentInfo):
        upiID = paymentInfo['upiId']
        upiPin = paymentInfo['upiPin']
        amount = paymentInfo['amount']
        
        return UPIPaymentManager(upiID, upiPin, amount)
    
    def getCardPaymentManager(self, paymentInfo):
        bankName = paymentInfo['bankName']
        cardName = paymentInfo['cardName']
        pin = paymentInfo['pin']
        amount = paymentInfo['amount']
        
        return CardBasedPaymentManager(bankName, cardName, pin, amount)
    
    def getCashOnDeliveryPaymentManager(self, paymentInfo):
        amount = paymentInfo['amount']
        
        return CashOnDeliveryPaymentManager(amount)
        